In [0]:
# Import all libraries
import tensorflow as tf
import tensorflow_datasets as td
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil

print(tf.__version__)

# Define variables/hyperparameters
eager_execution = False
batch_size = 32
nclasses = 10
num_epochs = 1
# This should ideally depend on number of records in a dataset
# But in Tensorflow 1.x there does not seem to be many good ways of determining it on the go
steps_per_epoch = 25000 

# To execute eagerly or not?
if eager_execution == True:
  # Enable eager execution
  tf.enable_eager_execution()

  # Checking eager execution
  a = tf.constant([[1, 2],
                   [3, 4]])
  b = tf.constant([[1, 2],
                  [3, 4]])
  c = tf.add(a, b)
  print(c)

W0806 04:52:54.884473 140281936885632 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



1.14.0


In [0]:
# Load MNIST Dataset
#td.list_builders() # To check all available datasets
mnist_dataset = td.load(name = 'mnist', split = 'train')

W0806 04:52:55.257077 140281936885632 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [0]:
# Add the batch dimension to our dataset. Tensorflow expects Conv2D to have dimesions like this: (batch, height, width, channels)
# So, induce the batch dimension
def add_dimension(dataset_row):
  dataset_row['image'] = tf.image.convert_image_dtype(dataset_row['image'], dtype = tf.float32)
  dataset_row['image'] = tf.expand_dims(dataset_row['image'], axis = 0)
  
  dataset_row['label'] = tf.one_hot(dataset_row['label'], depth = 10, axis = 0)

  return dataset_row

# Larger function to accommodate all preprocessing
def preprocess(dataset):
  dataset = dataset.map(add_dimension)
  dataset = dataset.shuffle(buffer_size = 10 * batch_size)
  dataset = dataset.repeat(count = num_epochs)
  
  return dataset

In [0]:
# Dataset ready for training
mnist_dataset = preprocess(mnist_dataset)

In [0]:
# Build model
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.InputLayer(input_shape = [28, 28, 1]))
model.add(tf.keras.layers.Conv2D(filters = 10, kernel_size = 3, padding = 'same', activation = tf.nn.relu))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2))
model.add(tf.keras.layers.Conv2D(filters = 20, kernel_size = 3, padding = 'same', activation = tf.nn.relu))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation = tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate = 0.25))
model.add(tf.keras.layers.Dense(10, activation = tf.nn.softmax))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:
# Check if Eager Execution is enabled or not
if tf.executing_eagerly() == True:
  print('Yes')
else:
  print('No')

No


In [0]:
#https://github.com/tensorflow/tensorflow/issues/26658
if tf.executing_eagerly() == True:
  mnist_iterator = iter(mnist_dataset)
  model.fit(x = next(mnist_iterator)['image'], y = next(mnist_iterator)['label'], batch_size = batch_size, epochs = num_epochs, steps_per_epoch = steps_per_epoch)
else:
  # make_one_shot_iterator() is DEPRECATED in TF 2.0
  mnist_iterator = mnist_dataset.make_one_shot_iterator()
  model.fit(x = mnist_iterator.get_next()['image'], y = mnist_iterator.get_next()['label'], batch_size = batch_size, epochs = num_epochs, steps_per_epoch = steps_per_epoch)

25000/25000 [==============================] - 117s 5ms/step - loss: 2.3031 - acc: 0.1091


In [0]:
# Load Test Dataset
mnist_test = td.load(name = 'mnist', split = 'test')
mnist_test = preprocess(mnist_test)

In [0]:
if tf.executing_eagerly() == True:
  mnist_iterator = iter(mnist_test)
  metrics = model.evaluate(x = next(mnist_iterator)['image'], y = next(mnist_iterator)['label'], batch_size = batch_size, steps = None)
else:
  # make_one_shot_iterator() is DEPRECATED in TF 2.0
  mnist_iterator = mnist_test.make_one_shot_iterator()
  metrics = model.evaluate(x = mnist_iterator.get_next()['image'], y = mnist_iterator.get_next()['label'], batch_size = batch_size, steps = 2500)

In [0]:
metrics

In [0]:
model.predict(mnist_iterator.get_next()['image'], steps = 1)

------------------------------------------------------------------------------MISCELLANEOUS----------------------------------------------------------------------

In [0]:
# Check info about this dataset
mnist_dataset_builder = td.builder('mnist')
print(mnist_dataset_builder.info)